# Custom MLflow Python Function 
### `ESM2 Protein Embedding Transformer`


<!-- # external ref https://huggingface.co/blog/AmelieSchreiber/protein-optimization-and-design -->

### Install Required Dependencies / Path Specs.

In [0]:
import sys
print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [0]:
%sh python --version

Python 3.11.11


In [0]:
# Create the requirements.txt file
requirements = """
torch==2.1.0
transformers==4.34.0
accelerate==0.23.0
cloudpickle==3.1.1
"""
catalog_name = "<your_catalog>"  
schema_name = "<your_schema>"  
volume_name = "<your_volume>"  

requirements_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/requirements.txt"

# Save the requirements.txt file to UC volumes
with open(requirements_path, "w") as f:
    f.write(requirements)

In [0]:
%pip install -r {requirements_path}

  Obtaining dependency information for torch==2.1.0 from https://files.pythonhosted.org/packages/5b/13/fcabc86948f9e89b62a538670720f8589d63f93d3f4f3d172236a98e70f8/torch-2.1.0-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for transformers==4.34.0 from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/121.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 5.2 MB/s eta 0:00:00
  Obtaining dependency information for accelerate==0.23.0 from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpickle==3.1.1 from https://files.pythonhosted.org/packages/7e/e8/64c37fadfc2816a7701fa8a6ed8d87327c7d54eacfbfb6edab14a2f2be75/cloudpickle-3.1.1-py3-non

In [0]:
dbutils.library.restartPython() 

Re-specify paths post kernal restart... 

In [0]:
catalog_name = "<your_catalog>"  
schema_name = "<your_schema>"  
volume_name = "<your_volume>"  
requirements_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/requirements.txt"

### Define, Log and Register Hugging Face Transformer as MLflow Custom PyFunc

In [0]:
# Custom PyFunc wrapper for a large transformer model
import mlflow
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer, logging
import shutil
import pandas as pd
from mlflow.models.signature import infer_signature

# Set the logging level to ERROR to disable verbose messages
# logging.set_verbosity_error()

# Define a Custom wrapper class for the transformer model
class ESMWrapper(mlflow.pyfunc.PythonModel):
    # load_context: Loads the model and tokenizer from the provided artifacts and sets up the device (CPU or GPU).
    def load_context(self, context):
        # Load ESM model from saved files in the artifact
        # self.artifacts = context.artifacts

        # Determine the device to use (GPU if available, otherwise CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the tokenizer from the provided artifacts
        self.tokenizer = AutoTokenizer.from_pretrained(context.artifacts["tokenizer"])
        # Load the model from the provided artifacts
        self.model = AutoModelForMaskedLM.from_pretrained(context.artifacts["model"])
        # Move the model to the appropriate device (GPU or CPU)
        self.model.to(self.device)
        # Set the model to evaluation mode
        self.model.eval()

        # Set special tokens if they are not set; ensure that the beginning-of-sequence (bos_token) and separator (sep_token) tokens are set. If they are not already set, they are assigned the value of the cls_token (classification token).
        if self.tokenizer.bos_token is None:
            self.tokenizer.bos_token = self.tokenizer.cls_token
        if self.tokenizer.sep_token is None:
            self.tokenizer.sep_token = self.tokenizer.cls_token
    
    # Define the predict function which takes input sequences, tokenizes them, runs them through the model, and returns the embeddings
    def predict(self, context, model_input):
        protein_sequences = model_input["sequences"]
        results = []
        
        # Process each sequence
        for seq in protein_sequences:
            inputs = self.tokenizer(seq, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model(**inputs, output_hidden_states=True)
                
            # Process outputs as needed for your application
            embeddings = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
            results.append(embeddings)
            
        return results

# Log the model with explicit dependencies
with mlflow.start_run():
    # Download and Save Model Components:
    model_name = "facebook/esm2_t33_650M_UR50D" #https://huggingface.co/facebook/esm2_t33_650M_UR50D
    
    # Specifies the model name and paths to save the model and tokenizer.
    model_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/tmp_model"
    tokenizer_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/tmp_tokenizer"
    
    # Download the pre-trained model and tokenizer from Hugging Face.
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Saves the model and tokenizer to the specified paths.
    model.save_pretrained(model_path, safe_serialization=False)
    tokenizer.save_pretrained(tokenizer_path)
    
    # Define conda env with necessary dependencies
    conda_env = {
        "channels": ["defaults", "conda-forge", "pytorch"],
        "dependencies": [
            "python=3.11", # compute 15.4LTSMLR 
            "pip>=22.0.4",
            {"pip": [
                "torch==2.1.0", 
                "transformers==4.34.0",
                "accelerate==0.23.0",
                "cloudpickle==3.1.1", #compute 15.4LTSMLR    
            ]}
        ],
        "name": "esm_env"
    }
    
    # Create a sample input DataFrame to infer the input and output signature of the model.
    sample_input = pd.DataFrame({"sequences": ["MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"]})
    
    ## Initialize the wrapper and load the context manually for signature inference

    # Initialize an instance of the ESMWrapper class.
    esm_wrapper = ESMWrapper()
    # Manually set the tokenizer and model for the wrapper.
    esm_wrapper.tokenizer = tokenizer
    esm_wrapper.model = model
    # Determine the device (GPU or CPU) and moves the model to the appropriate device.
    esm_wrapper.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    esm_wrapper.model.to(esm_wrapper.device)
    # Set the model to evaluation mode.
    esm_wrapper.model.eval()
    
    # Use the wrapper to predict the output for the sample input
    sample_output = esm_wrapper.predict(None, sample_input)
    # Infer the input and output signature of the model using the sample input and output
    signature = infer_signature(sample_input, sample_output)
    
    # Log the model with MLflow, including the artifacts (model and tokenizer paths), Conda environment, signature, and input example.
    mlflow.pyfunc.log_model(
        artifact_path="esm_model",
        python_model=ESMWrapper(),
        artifacts={
            "model": model_path,
            "tokenizer": tokenizer_path,
            "requirements": requirements_path ## volumes path to requirements.txt file 
        },
        conda_env=conda_env,
        signature=signature,
        input_example=sample_input,
        # Register the model with a specified name.
        registered_model_name=f"{catalog_name}.{schema_name}.esm_protein_model"
    )

Uploading artifacts:   0%|          | 0/17 [00:00<?, ?it/s]

Uploading /local_disk0/user_tmp_data/root/tmpmoj5tcft/model/artifacts/tmp_model/pytorch_model.bin:   0%|      …

Registered model 'mmt_demos.dependencies.esm_protein_model' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/17 [00:00<?, ?it/s]

Uploading /local_disk0/user_tmp_data/root/tmpmoj5tcft/model/artifacts/tmp_model/pytorch_model.bin:   0%|      …

Created version '9' of model 'mmt_demos.dependencies.esm_protein_model'.


In [0]:
sample_output

[array([[ 0.06840793,  0.0689078 , -0.04763591, ..., -0.01513798,
         -0.12106971,  0.0301559 ]], dtype=float32)]

In [0]:
# [9xz5z] [2025-04-07 16:15:05 +0000] 2025/04/07 16:15:05 WARNING mlflow.pyfunc: The version of CloudPickle that was used to save the model, `CloudPickle 2.2.1`, differs from the version of CloudPickle that is currently running, `CloudPickle 3.1.1`, and may be incompatible

In [0]:
import mlflow
model_version = 9
model_uri = f"models:/{catalog_name}.{schema_name}.esm_protein_model/{model_version}"
dependencies = mlflow.pyfunc.get_model_dependencies(model_uri)
print(dependencies)

2025/05/23 04:23:14 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /local_disk0/user_tmp_data/root/tmpkk7epkw6/requirements.txt'.


/local_disk0/user_tmp_data/root/tmpkk7epkw6/requirements.txt


In [0]:
!cat $dependencies

mlflow==2.19.0
torch==2.1.0
transformers==4.34.0
accelerate==0.23.0
cloudpickle==3.1.1

### Specify endpoint configs and deploy UC registered mlflow pyfunc model as endpoint

In [0]:
from mlflow.tracking import MlflowClient

def get_latest_model_version(model_name):
    mlflow_client = MlflowClient(registry_uri="databricks-uc")
    latest_version = 1
    for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
        version_int = int(mv.version)
        if version_int > latest_version:
            latest_version = version_int
    return latest_version

def get_model_uri(model_name):
  return f"models:/{model_name}/{get_latest_model_version(model_name)}"

In [0]:
workload_type = "GPU_SMALL"
workload_size = "Small"

registered_model_name = f"{catalog_name}.{schema_name}.esm_protein_model"
latest_model_version = get_latest_model_version(registered_model_name)
general_model_name = f"esm_protein_model-{latest_model_version}" 

endpoint_base_name = "esm_protein_model_endpoint"
endpoint_name = f"{endpoint_base_name}-mmt-mlflowsdk" # endpoint name

print(f"catalog: {catalog_name}",  "schema:", {schema_name}, 
      workload_type, workload_size, "general_model_name:", general_model_name, 
      "registered_model_name:", registered_model_name, "latest_model_version:", 
      latest_model_version, "endpoint_name:", endpoint_name
      )

catalog: mmt_demos schema: {'dependencies'} GPU_SMALL Small general_model_name: esm_protein_model-9 registered_model_name: mmt_demos.dependencies.esm_protein_model latest_model_version: 9 endpoint_name: esm_protein_model_endpoint-mmt-mlflowsdk


In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")

# Define the full API request payload
endpoint_config = {
    "name": general_model_name,
    "served_models": [
        {                
            "model_name": registered_model_name,
            "model_version": latest_model_version,
            "workload_size": workload_size,  # defines concurrency: Small/Medium/Large
            "workload_type": workload_type,  # defines compute: GPU_SMALL/GPU_MEDIUM/GPU_LARGE
            "scale_to_zero_enabled": True
        }
    ],
    "traffic_config": {
        "routes": [
            {
                "served_model_name": general_model_name,
                "traffic_percentage": 100
            }
        ]
    },
    "auto_capture_config": {
        "catalog_name": catalog_name,
        "schema_name": schema_name,
        # "table_name_prefix": endpoint_base_name,
        "enabled": True
    },
    "tags": [
        {"key": "project", "value": "esm_protein_model"},
        {"key": "team", "value": "ml-team"},
        {"key": "removeAfter", "value": "2025-12-31"}
    ]
}

# Create or update the endpoint
try:
    # Check if endpoint exists
    existing_endpoint = client.get_endpoint(endpoint_name)
    print(f"Endpoint {endpoint_name} exists, updating configuration...")
    client.update_endpoint(endpoint_name, endpoint_config)
except Exception as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print(f"Creating new endpoint {endpoint_name}...")
        client.create_endpoint(endpoint_name, endpoint_config)
    else:
        raise

Endpoint esm_protein_model_endpoint-mmt-mlflowsdk exists, updating configuration...


In [0]:
from databricks.sdk import WorkspaceClient
import time

# Initialize the Databricks client
client = WorkspaceClient()

# Check if the endpoint already exists
existing_endpoints = client.serving_endpoints.list()
if endpoint_name in [endpoint.name for endpoint in existing_endpoints]:
    print(f"Endpoint with name '{endpoint_name}' already exists.")
else:
    # Convert endpoint_config to the expected format if necessary
    class EndpointConfig:
        def __init__(self, config_dict):
            self.config_dict = config_dict

        def as_dict(self):
            return self.config_dict

    # Create the serving endpoint
    response = client.serving_endpoints.create(
        name=endpoint_name,
        config=EndpointConfig(endpoint_config)
    )

    if response:
        print("Serving endpoint created successfully.")
    else:
        print(f"Failed to create serving endpoint: {response}")

# Check the status of the endpoint
max_retries = 6  # Number of times to check (6 times for 1 hour)
retry_interval = 600  # 10 minutes in seconds

for attempt in range(max_retries):
    status_response = client.serving_endpoints.get(endpoint_name).as_dict() #
    if status_response:
        state = status_response['state']
        ready = state.get('ready', False)
        config_update = state.get('config_update', '')
        message = state.get('message', '')
        pending_config = status_response.get('pending_config', {})
        served_entities = pending_config.get('served_entities', [])
        served_entity_state = served_entities[0].get('state', 'Unknown') if served_entities else "Unknown"
        
        if ready and config_update != 'IN_PROGRESS':
            print("Serving endpoint is ready.")
            break
        else:
            print(state)
            print(f"Serving endpoint state: {message}")
            print(f"Served entity state: {served_entity_state}")
            print("Waiting for the serving endpoint config_updates to be ready...")
            time.sleep(retry_interval)  # Wait for 10 minutes before checking again
    else:
        print(f"Failed to get endpoint status: {status_response}")
        break
else:
    print("Endpoint config_update is still not ready after the maximum number of check retries. Please review the Serving UI to see if there are errors or issues.")

Endpoint with name 'esm_protein_model_endpoint-mmt-mlflowsdk' already exists.
Serving endpoint is ready.


### Test model endpoint inferencing

In [0]:
import logging
from databricks.sdk import WorkspaceClient

# Define a logger
_logger = logging.getLogger(__name__)

# Define a workspace client with default notebook authentication
user_authenticated_client = WorkspaceClient()

# TEST 
try:
    user_authenticated_client.serving_endpoints.query(endpoint_name, input="")
except Exception as e:
    _logger.debug("Skipping Model Serving Endpoint due to no permissions")


# Sample input
input_data = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}
    ]
}

query_response = user_authenticated_client.serving_endpoints.query(
    endpoint_name,                  
    inputs=input_data["dataframe_records"]
)

query_response.predictions 

[[[0.06840793788433075,
   0.06890784204006195,
   -0.04763609543442726,
   0.02628912404179573,
   -0.08527260273694992,
   0.039011821150779724,
   -0.0711316242814064,
   0.20650677382946014,
   0.07555650174617767,
   0.02792150154709816,
   0.010508893989026546,
   0.05739198997616768,
   0.14937055110931396,
   0.13881385326385498,
   -0.05469195544719696,
   0.06682949513196945,
   -0.016842197626829147,
   0.04010609909892082,
   -0.057004936039447784,
   -0.06419392675161362,
   0.08771298080682755,
   -0.03626326099038124,
   -0.00040011206874623895,
   0.08976373821496964,
   0.043842874467372894,
   -0.05397772043943405,
   -0.06927525997161865,
   -0.06238190084695816,
   -0.05292431637644768,
   -0.15746259689331055,
   -0.05517962947487831,
   -0.056654464453458786,
   0.07234666496515274,
   -0.032119181007146835,
   -0.012918336316943169,
   0.006699382793158293,
   -0.10783190280199051,
   0.032689131796360016,
   0.09280376881361008,
   -0.023149622604250908,
   0.03

### Format and test input_data on served model endpoint UI

In [0]:
input_data = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}
    ]
}

input_data

{'dataframe_records': [{'sequences': 'MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG'}]}

In [0]:
import json
json.dumps(input_data)

'{"dataframe_records": [{"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}]}'

In [0]:
# https://huggingface.co/blog/AmelieSchreiber/protein-optimization-and-design 

# MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE

In [0]:
input_data2 = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"},
        {"sequences": "MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE"}
    ]
}

# input_data2 
json.dumps(input_data2)

({'dataframe_records': [{'sequences': 'MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG'},
   {'sequences': 'MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE'}]},
 '{"dataframe_records": [{"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}, {"sequences": "MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE"}]}')

In [0]:
query_response2 = user_authenticated_client.serving_endpoints.query(
    endpoint_name,                  
    inputs=input_data2["dataframe_records"]
)

query_response2#.predictions

QueryEndpointResponse(choices=[], created=None, data=[], id=None, model=None, object=None, predictions=[[[0.06840793788433075, 0.06890784204006195, -0.04763609543442726, 0.02628912404179573, -0.08527260273694992, 0.039011821150779724, -0.0711316242814064, 0.20650677382946014, 0.07555650174617767, 0.02792150154709816, 0.010508893989026546, 0.05739198997616768, 0.14937055110931396, 0.13881385326385498, -0.05469195544719696, 0.06682949513196945, -0.016842197626829147, 0.04010609909892082, -0.057004936039447784, -0.06419392675161362, 0.08771298080682755, -0.03626326099038124, -0.00040011206874623895, 0.08976373821496964, 0.043842874467372894, -0.05397772043943405, -0.06927525997161865, -0.06238190084695816, -0.05292431637644768, -0.15746259689331055, -0.05517962947487831, -0.056654464453458786, 0.07234666496515274, -0.032119181007146835, -0.012918336316943169, 0.006699382793158293, -0.10783190280199051, 0.032689131796360016, 0.09280376881361008, -0.023149622604250908, 0.035779647529125214,

### Batch processing using:
1. MLflow UC registered model    
2. Served model endpoint with `ai_query()`

In [0]:
## actually slower... 

import mlflow.pyfunc
from pyspark.sql.functions import col, flatten

# Load the registered model as a Spark UDF
model_name = "mmt_demos.dependencies.esm_protein_model"
model_version = 8
model_uri = f"models:/{model_name}/{model_version}"
model_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

# Example Spark DataFrame with sequences
df = spark.createDataFrame([
    ("MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG",),
    ("MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE",)
], ["sequences"])

# Apply the model UDF to the DataFrame
df_result = df.withColumn("predictions", 
                          flatten(model_udf(col("sequences"))) 
                         )

# Display the result
display(df_result)

2025/05/23 05:11:42 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.13.1, required: mlflow==2.19.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/05/23 05:11:42 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2025/05/23 05:11:44 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


sequences predictions MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG List(0.06840838, 0.06890764, -0.047635622, 0.026289972, -0.08527244, 0.03901175, -0.07113164, 0.20650642, 0.075556405, 0.027921386, 0.010509114, 0.05739173, 0.14937013, 0.13881384, -0.05469197, 0.06682936, -0.016842175, 0.040105406, -0.057004802, -0.06419331, 0.08771298, -0.036262814, -3.9931867E-4, 0.08976264, 0.043843046, -0.05397754, -0.06927525, -0.062382814, -0.052924484, -0.15746221, -0.055178527, -0.056654777, 0.07234708, -0.0321189, -0.01291865, 0.006699102, -0.10783113, 0.03268929, 0.09280348, -0.023149347, 0.035779912, 0.051875338, 0.104161516, 0.036204312, -0.020499017, -0.13204771, -0.025253847, 0.035619494, 0.0059802984, -0.055136178, -2.174298E-4, 0.019700186, -0.023575386, -0.009352514, -0.059250142, 0.07128532, 0.050104108, 0.12734567, -0.05880791, -0.09344798, 0.09654929, 0.05974936, -0.042374842, -0.04004159, -0.0013047893, 0.012204811, 0.12572199, -0.14124285, -0.15539838, -0.061605018, 0.019362621, -0.08097379, -0.03506991, 0.073918544, -0.040058047, -0.059401672, -0.089189276, 0.060319133, -0.02022091, -0.0056214835, 0.048524518, 0.018278817, 0.0010253814, 0.09307837, 0.02712849, -0.09231533, -0.09629869, 0.042844933, -0.018366043, 0.048099276, -0.029144188, -0.03499457, -0.17611751, 0.010257972, -0.08848524, -0.043883998, 0.10064373, -0.0339738, -0.11548835, -0.036989503, 0.043328196, 0.03603001, 0.13751356, -0.11825175, -0.038717885, 0.036287323, -0.1414252, 0.03292288, -0.035258584, -0.00188574, -0.14186962, 0.05842782, -0.0035114898, -0.08142995, 0.019648658, 0.114132404, -0.055711523, -0.080169156, -0.0075739776, 0.16955638, 0.12441348, -0.017835954, 0.02696262, 0.05134423, -0.08907049, -0.041796487, 0.045411758, 0.035128277, -0.12006074, 0.05087772, 0.069432214, -0.02886493, -0.058210846, 0.07436322, 0.045044243, 0.07949377, -0.06659904, -0.15419693, 0.03215673, -0.020381082, 0.04938292, 0.15750217, -0.11478337, 0.04987665, -0.017715475, 0.09344613, 0.019007338, -2.24989E-4, -0.061440263, 0.09784706, 0.0042846324, 0.053707123, 0.009988715, -0.1501442, 0.14166497, 0.097513594, 0.10253226, 0.0053085494, 0.13271773, -0.059877925, 0.022200098, 0.1171344, -0.011334359, -0.02471003, -0.023027629, -0.016131802, -0.046666343, 0.01462398, 0.084212884, 0.066926666, 0.025924167, 0.022272918, 0.064776175, -0.011332101, -0.08613981, 0.050320048, -0.004136186, -0.04456413, 0.15151908, -0.0170159, -0.0861461, -0.07683348, 0.02028802, 0.001780227, 0.01900675, -0.011663814, 0.03276826, -0.10094982, 0.009796024, 0.12488069, 0.008370548, -0.026330061, -0.0040273387, -0.02120983, 0.039961275, 0.010805782, -0.025436904, -0.00964904, -0.0255755, 0.013366074, 0.06080822, 0.051675722, -0.11963483, 0.017710077, 0.047936052, -0.04474221, 0.11451297, 0.009928729, -0.124536216, -0.12566915, 0.096510306, -0.0019427296, 0.013614252, -0.025503784, -0.10239096, -0.06484273, 0.19323403, 0.048344523, 0.10714174, 0.030915944, -0.041921254, 0.066263035, 0.065261476, -0.026423842, 0.058818515, -0.026068065, 0.15361162, 0.17265104, -0.19219716, -0.04913991, 0.073286876, -0.013871964, -0.11204328, 0.0029997027, -4.79073, 0.028447082, 0.015687698, -0.08928967, 0.052844014, -0.25598684, 0.017638208, 0.047088012, -8.848283E-4, 0.0325487, -0.0013405575, -0.080995776, 0.014094039, 0.054281123, 0.0088950405, -0.052126482, -0.019123573, -0.031808544, -0.051905006, 0.09669072, 0.16983812, 0.019641241, 0.015870104, 0.059061587, -0.04634817, -0.100031026, 0.106872134, 0.006723211, -0.019107342, 0.1045846, 0.15299396, -0.00644266, -0.08347011, 0.040234137, -0.10229685, -0.029127365, 0.010929563, -0.057262115, 0.018474717, 0.019283276, 0.07076602, 0.14076099, -0.06272994, -0.10837574, -0.047028914, -0.11437456, -0.028760208, -0.21158202, 0.043966196, 0.02046202, -0.105107866, -0.16485128, 0.06556968, -0.008672744, -5.862799E-4, 0.02903684, -0.12931895, -0.007842204, -0.16524528, -0.020395488, -0.06659208, 0.01604682, -0.03211241, -0.04088715, 0.11383863, -0.005290725, -0.0025284183, 0.

In [0]:
from pyspark.sql.functions import col, struct
from pyspark.sql import functions as F

# Use/define endpoint name
# endpoint_name = "esm_protein_model_endpoint-mmt-mlflowsdk"

# Example Spark DataFrame with sequences
df = spark.createDataFrame([
    ("MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG",),
    ("MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE",),
    ("MAPLFKTYVLKLWVAGNTPN",) # randomly generated mutation
], ["sequences"])

# Use ai_query function to query the endpoint
df_with_predictions = df.withColumn(
    "prediction",
    # F.expr(f"ai_query('{endpoint_name}', struct(sequences))")
    F.expr(f"ai_query('{endpoint_name}', sequences)[0]") 
)

# Display the result
display(df_with_predictions)

sequences prediction MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG List(0.06840794, 0.06890784, -0.047636095, 0.026289124, -0.0852726, 0.03901182, -0.071131624, 0.20650677, 0.0755565, 0.027921502, 0.010508894, 0.05739199, 0.14937055, 0.13881385, -0.054691955, 0.066829495, -0.016842198, 0.0401061, -0.057004936, -0.06419393, 0.08771298, -0.03626326, -4.0011207E-4, 0.08976374, 0.043842874, -0.05397772, -0.06927526, -0.0623819, -0.052924316, -0.1574626, -0.05517963, -0.056654464, 0.072346665, -0.03211918, -0.012918336, 0.006699383, -0.1078319, 0.03268913, 0.09280377, -0.023149623, 0.035779648, 0.051875137, 0.10416208, 0.036204338, -0.020499412, -0.13204803, -0.02525354, 0.03561925, 0.0059801224, -0.055135846, -2.1745828E-4, 0.019699689, -0.023576265, -0.00935201, -0.059250455, 0.071284935, 0.050104868, 0.12734553, -0.05880816, -0.09344792, 0.096549176, 0.059749223, -0.042374775, -0.040042143, -0.0013046728, 0.012205901, 0.12572211, -0.14124347, -0.1553979, -0.061606538, 0.019362643, -0.08097422, -0.035069484, 0.07391929, -0.040058672, -0.059401195, -0.08918937, 0.06031931, -0.020221155, -0.0056216037, 0.048523802, 0.018279234, 0.0010251419, 0.09307883, 0.027128117, -0.09231592, -0.09629823, 0.042844374, -0.018366234, 0.04810005, -0.02914399, -0.03499469, -0.17611836, 0.010257617, -0.08848514, -0.043884527, 0.100643806, -0.033974346, -0.1154887, -0.0369891, 0.04332762, 0.03602978, 0.13751431, -0.11825207, -0.03871867, 0.03628723, -0.14142665, 0.032923404, -0.03525817, -0.001885201, -0.14187008, 0.05842831, -0.0035118521, -0.08143016, 0.019648846, 0.11413273, -0.05571138, -0.08016915, -0.0075730034, 0.16955711, 0.12441398, -0.017836358, 0.026962243, 0.051343814, -0.089070395, -0.04179708, 0.045412768, 0.035129357, -0.12006119, 0.05087762, 0.069432355, -0.02886489, -0.058211014, 0.074363135, 0.045044165, 0.0794948, -0.06659776, -0.15419686, 0.032156974, -0.020381687, 0.049383298, 0.15750237, -0.11478398, 0.0498765, -0.01771576, 0.093446225, 0.019006833, -2.2464048E-4, -0.061441038, 0.097847626, 0.004283281, 0.053706992, 0.009988454, -0.15014431, 0.14166535, 0.09751435, 0.102532476, 0.005308849, 0.13271774, -0.059878107, 0.0221993, 0.117135644, -0.011334801, -0.024710264, -0.023027897, -0.016130574, -0.046666972, 0.014624121, 0.08421323, 0.06692702, 0.025923515, 0.0222732, 0.06477591, -0.0113320025, -0.08613905, 0.050319705, -0.004136385, -0.044563565, 0.1515195, -0.017016126, -0.0861463, -0.07683315, 0.020288685, 0.0017801934, 0.019006534, -0.011663531, 0.032768846, -0.100950435, 0.009796022, 0.124880776, 0.008370491, -0.026330657, -0.004027821, -0.021209696, 0.039961133, 0.010805228, -0.025436739, -0.009648199, -0.025574747, 0.013366432, 0.06080755, 0.051675554, -0.11963538, 0.01771051, 0.047935985, -0.04474174, 0.11451311, 0.009928426, -0.12453619, -0.12566896, 0.096510224, -0.001942677, 0.013614137, -0.025503954, -0.10239078, -0.06484347, 0.193234, 0.04834418, 0.10714206, 0.03091593, -0.041921344, 0.06626271, 0.065262206, -0.026423577, 0.058818113, -0.026069207, 0.15361166, 0.17265041, -0.19219702, -0.04913946, 0.073286936, -0.013871239, -0.11204348, 0.0029985793, -4.790734, 0.028446784, 0.015687099, -0.08928978, 0.05284358, -0.25598767, 0.017638793, 0.04708858, -8.8494027E-4, 0.032549053, -0.0013408883, -0.08099685, 0.014094401, 0.05428197, 0.008895735, -0.052127443, -0.019123822, -0.03180861, -0.05190555, 0.09669127, 0.16983789, 0.019639824, 0.01587091, 0.05906193, -0.046349242, -0.10003088, 0.10687288, 0.0067231646, -0.019107644, 0.10458537, 0.15299408, -0.006442845, -0.08347113, 0.040234894, -0.10229683, -0.029127132, 0.010930232, -0.05726208, 0.018474119, 0.019283976, 0.070765875, 0.14076112, -0.06273048, -0.108375944, -0.047028888, -0.1143751, -0.028760418, -0.21158293, 0.043967094, 0.020462122, -0.10510801, -0.16485217, 0.06556936, -0.008671817, -5.8623287E-4, 0.029037379, -0.1293192, -0.007842674, -0.16524564, -0.020396668, -0.06659272, 0.016046695, -0.032112647, -0.040887475, 0.113839366, -0.0052908086, -0.002528982, 0.0442